# 変量bloodクレンジング

変量bloodについてクレンジングを実施します．
この変量に対するクレンジングは，変量genderに対するプロセスとほぼ同じです．

先ず，前提として次のプログラムを実施します．

- ライブラリーの搬入
- CSVファイルを読み込んでデータフレームを生成
- 全角半角変換ツールの準備
- 小数点付数を判別する正規表現のコンパイル


In [1]:
import re
import unicodedata
import pandas as pd

In [2]:
df = pd.read_csv('original_body_data.csv')
df.set_index('person',inplace=True)

In [3]:
floatCheck = re.compile(r'^\s*([+-]?(\d+\.?\d*|\.\d+))\s*$')

*****
## データフレームの基礎情報

まずはデータフレームの行数を確認します．
そして，データフレームの一部を表示します．

```Python
len(df)
df.head(5)
```

In [4]:
len(df)

200

In [5]:
df.head(5)

,height,weight,age,gender,blood
person,,,,,
p001,157.7,56.8,20,Male,B
p002,169.7,58.1,53,Female,A
p003,160.6,96.8,22,Male,O
p004,'155.8',,,Female,O
p005,115.5,22.5,7,Male,NaN


*****
## 変量genderの概要

変量の概要は<font color=green>describe()</font>メソッドを発行することによって得ることができます．

```Python
df.blood.describe()
```

In [6]:
df.blood.describe()

count     190
unique     13
top         A
freq       67
Name: blood, dtype: object

describe()メソッドの出力の最後の行に注目します．

> 
```
Name: blood, dtype: object
```

これは，df.bloodの各データのデータ型がobject型であることを示しています．
各要素のデータ型を調べるために，リスト内包表記とtype()関数を組み合わせます．そして，その結果をset()関数で重複を除いたデータ型の取得します．

```Python
set([type(x) for x in df.blood])
```

In [7]:
set([type(x) for x in df.blood])

{str, float}

df.bloodの各要素はstr型とfloat型が混在していることが分かりました．
そこで，float型のデータの値を確認します．

```Python
[x for x in df.blood if type(x)==float]
```

In [8]:
[x for x in df.blood if type(x)==float]

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

この出力を見るとfloat型の値は全て欠損値であることが分かります．


## カテゴリー値の確認

df.bloodの各要素の文字列で，その重複を除いた値を確認します．



```Python
set([x for x in df.blood if type(x)==str])
```

In [9]:
set([x for x in df.blood if type(x)==str])

{' AB ',
 ' o ',
 'A',
 'A ',
 'AB',
 'Ab',
 'B',
 'O',
 'o',
 '\u3000Ａ',
 '\u3000Ａ\u3000',
 'Ｏ',
 'ａ'}

df.bloodの有効な値として想定しているものは['A','B','O','AB']です．
しかし，この出力を見ると次の問題があります．

- 全角文字がある．
- 大文字小文字が混在している．
- 文字列の前後に空白文字がある．

これらの課題を解消するようにクレンジングを実施します．

## 全角から半角への変換

全角文字の例として，データ「\u3000Ａ\u3000」について，全角文字を半角に変換します．

```Python
'\u3000３９\u3000'.strip("'").translate(doubleToSingle)
```

In [10]:
unicodedata.normalize('NFKC','\u3000Ａ\u3000')

' A '

df.bloodの全データについて全角を半角に変換した結果の値の集合を表示します．
これはリスト内包表記によって半角に変換したリストからset()関数によって重複を除いた値を求めます．

```Python
set([unicodedata.normalize('NFKC',x) if type(x)==str else x for x in df.blood])
```

In [11]:
set([unicodedata.normalize('NFKC',x) if type(x)==str else x for x in df.blood])

{nan, 'Ab', 'O', 'o', ' AB ', 'a', 'B', 'AB', 'A ', ' A ', ' A', ' o ', 'A'}

## 空白の削除

文字列の前後にある空白を削除します．
空白の削除は<font color=green>strip()</font>メソッドを使用します．
確認のために ' AB ' について実施してみます．

```Python
' AB '.strip()
```

In [12]:
' AB '.strip()

'AB'

全角を半角の変換した後に，前後の空白文字を削除します．
その結果で求まる値を重複を除いて表示します．

```Python
set([x.translate(doubleToSingle).strip() for x in df.blood if type(x)==str])
```

In [13]:
set([unicodedata.normalize('NFKC',x).strip() if type(x)==str else x for x in df.blood])

{nan, 'Ab', 'O', 'o', 'a', 'B', 'AB', 'A'}

## 小文字を大文字に統一

ここまでクレンジングした結果として文字列は血液型を示す値となりました．
しかし，大文字小文字の統制が取れていません．
そこで全ての小文字を大文字に変換します．
この処理は<font color=green>upper()</font>メソッドで実現します．
試しに一つの文字列を変換してみます．

```Python
'ab'.upper()
```

In [14]:
'ab'.upper()

'AB'

この処理も追加して，その結果の文字列の種類を表示します．

```Pythonn
set([unicodedata.normalize('NFKC',x).strip().upper() if type(x)==str else x for x in df.blood])
```

In [15]:
set([unicodedata.normalize('NFKC',x).strip().upper() if type(x)==str else x for x in df.blood])

{nan, 'O', 'B', 'AB', 'A'}

## データフレームでの処理

ここまでdf.bloodを取り出してリスト配列としてクレンジング処理を行ってきましたが，実際にはデータフレームの更新作業となります．
よって，データフレーム内でのクレンジングを行うプログラムを作成します．


### 文字列の整形

まず，df.bloodを文字列として綺麗な形に整えます．
その結果は新しい列blood2としてデータフレームに追加します．

```Python
df['blood2'] = [x.translate(doubleToSingle).strip().upper() if type(x)==str else x for x in df.blood]
```

In [16]:
df['blood2'] = [unicodedata.normalize('NFKC',x).strip().upper() if type(x)==str else x for x in df.blood]

### クレンジング済み変量gender2の確認

これで性別データについてのクレンジングが終了したので，その結果を<font color=green>describe()</font>メソッドで確認します．

```Python
df.blood2.describe()
```

In [17]:
df.blood2.describe()

count     190
unique      4
top         A
freq       71
Name: blood2, dtype: object

この出力により，190個の全てのデータが文字列になっていることが分かります．
行数は200行でしたので，残りのデータは数値として値が入っていない欠損値になっています．
欠損値以外のユニークな値は想定通り4個になりました．

df.bloodr2の取りうる値の種類を確認します．

```Python
set(df.blood2)
```

In [18]:
set(df.blood2)

{nan, 'O', 'B', 'AB', 'A'}

念のためにデータフレームの先頭の数行を表示します．
新たにblood2という列が追加されていることが確認できます．

In [19]:
df.head(4)

,height,weight,age,gender,blood,blood2
person,,,,,,
p001,157.7,56.8,20,Male,B,B
p002,169.7,58.1,53,Female,A,A
p003,160.6,96.8,22,Male,O,O
p004,'155.8',,,Female,O,O


これで，血液型のデータについてのクレンジングは完了です．

*****